In [1]:
# pip install tda-api

# from tda import auth, client
# import json

# import config_tda
# from tda import auth, client
# import json


# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_price_history('AAPL',
#         period_type=client.Client.PriceHistory.PeriodType.YEAR,
#         period=client.Client.PriceHistory.Period.TWENTY_YEARS,
#         frequency_type=client.Client.PriceHistory.FrequencyType.DAILY,
#         frequency=client.Client.PriceHistory.Frequency.DAILY)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))
# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_account(client.Client.Account.ORDERS)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))

In [36]:
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
from dotenv import load_dotenv
from web3 import Web3
from pathlib import Path
import json
load_dotenv()

import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
import time
import pandas as pd

from web3 import Web3
from pathlib import Path

import json
from io import StringIO

from dotenv import load_dotenv
load_dotenv()

True

In [3]:

# Set Alpca API key and secret key, then insantiate an API object
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECURITY_KEY")

tradeapi = REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [4]:
def get_ticker(ticker, start, end, tradeapi):
    """Executes the get_bars method from Alpaca Trade API"""
    return tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    
def grade_range(one_row_df):
    """Return the high and low of a single day"""
    return one_row_df[['high', 'low']].reset_index().drop(columns='timestamp').values

In [5]:
grade_range(get_ticker("SPY", "2022-07-26", "2022-07-26", tradeapi))

array([[394.06, 389.95]])

In [139]:
import yfinance as yf
data = yf.download(tickers='MSFT', period='5d', interval='5m')

[*********************100%***********************]  1 of 1 completed


In [140]:
data

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-08-01 09:30:00-04:00,277.820007,278.250000,276.743103,277.640015,277.640015,1236122
2022-08-01 09:35:00-04:00,278.000000,278.500000,277.690002,278.290009,278.290009,505005
2022-08-01 09:40:00-04:00,278.250000,278.899994,278.130005,278.239990,278.239990,345137
2022-08-01 09:45:00-04:00,278.000000,278.579987,277.822998,278.350006,278.350006,310118
2022-08-01 09:50:00-04:00,278.000000,278.959991,277.849915,278.739990,278.739990,412996
...,...,...,...,...,...,...
2022-08-04 15:50:00-04:00,283.160004,283.609985,283.160004,283.299988,283.299988,430927
2022-08-04 15:55:00-04:00,283.309998,283.799988,283.049988,283.649994,283.649994,676929
2022-08-05 09:30:00-04:00,279.149994,281.045013,278.679993,280.519989,280.519989,1467792


In [6]:
def verify_trade(trade_event):
    
    
    return verify_status

In [7]:
class Grader:
    # add event filters to Grader class 
    def __init__(self, trader_id, log_address):
        self.trader_id = trader_id
        self.log_address = log_address
        
    def load_contract(self):
        contract_address = self.log_address
        with open(Path('logger.json')) as f:
            logger_abi = json.load(f)
            
            self.contract = w3.eth.contract(
            address=contract_address,
            abi=logger_abi)
            
            self.new_trade_filter = self.contract.events.newTrade.createFilter(fromBlock="latest")
            self.close_trade_filter = self.contract.events.newCloseTrade.createFilter(fromBlock="latest")
            self.new_sub_filter = self.contract.events.newSub.createFilter(fromBlock="latest")
            
        return None
    
    def verify_trade(self):
        pass        
    

In [74]:
import atexit

In [103]:
import streamlit as st
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
import time
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path

import json
from io import StringIO

load_dotenv()


w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

def load_deployer_contract():
    contract_address = os.getenv("DEPLOYER_ADDRESS") 
    with open(Path('deployer.json')) as f:
            deployer_abi = json.load(f)

    deployer_contract = w3.eth.contract(
        address=contract_address,
        abi=deployer_abi
        )
    trader_dict = {}
    return deployer_contract, trader_dict

def load_contract_new(log_addr):
    contract_address = log_addr
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
        address=contract_address,
        abi=logger_abi
            )
    return logger_contract

def handle_new_log(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    

def handle_new_trade(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)

    
def exit_protocol(list_dfs):
    # Before this Joe, I tried zip then iter, list of tups then take n, df -- this works at least
    csv_names = ["logger.csv", "sub.csv", "all_trades.csv"]
    for df , fname in zip(list_dfs,csv_names):
        df.to_csv(Path(fname))
    
        
    
    

In [9]:
# Instantiate deployer and trader dictionary
deployer, trader_dict = load_deployer_contract()

deployer.address

'0x9725c52c23e7E871DC2CF19f20f101C6E80EeD29'

In [111]:
exit_protocol( [(logger_df), (sub_df), (all_trades_df)])

In [115]:
def load_database():
    # Instantiate dfs with columns that match event keys -- don't screw with these column names (unless you plan on changing the smart contract or add a pandas mapping inside append functions during event handling)

    # Check for existing data -- if it doesn't exist, then create new dataframes
    if not(os.path.exists("logger.csv")):
        no_storage = input("no csv storage found, would you like to start from scratch? (y/n)")
        if no_storage.strip() == "y"

            logger_df = pd.DataFrame(columns=(['TraderId','newLogAddress']))
            sub_df = pd.DataFrame(columns=(['TraderId', 'subNum', 'subAddr']))
            all_trades_df = pd.DataFrame(columns=(['TraderId',
                                                    'inputTraderAddress',
                                                    'tradeNum',
                                                    'inputOpen',
                                                    'inputSymbol',
                                                    'inputSize',
                                                    'inputEntryPriceEntryTime',
                                                    "inputExitPriceEntryTime",
                                                    'optionsData']))
        else:
            print("If you would like new events stored, please close the server and move these files into the server's directory: logger.csv, sub.csv, all_trades.csv")
            return 0 , 0 , 0
    
    else:
        logger_df = pd.DataFrame("logger.csv")
        sub_df = pd.DataFrame("sub.csv")
        all_trades_df = pd.DataFrame("all_trades.csv")
        
    return logger_df , sub_df , all_trades_df


In [137]:

# Create a filter to find new log contracts
new_log_filter = deployer.events.logCreated.createFilter(fromBlock="latest")
# Listen
print("Starting while loop...")
while True:
    
    # 1) Listen for new traders creating trading logs
    new_log_event = handle_new_log(new_log_filter.get_new_entries())
    if new_log_event:
        new_log = new_log_event[0]['args']
        print("--------------------------")
        print((new_log))
        print("--------------------------")
        # add the information to a Grader object stored at that trader's ID variable
        logger_df = logger_df.append(new_log,ignore_index=True)
        
        trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'], new_log['newLogAddress'])
        trader_dict[new_log['TraderId']].load_contract()
            
    
    # loop through each trader and look for new trades at their log's contract address -- here we're looking for new subs and trades across all log contracts the server has 
    # heard were created
    for key in trader_dict.keys():
        
        new_sub_event = handle_new_trade(trader_dict[key].new_sub_filter.get_new_entries())
        new_trade_event = handle_new_trade(trader_dict[key].new_trade_filter.get_new_entries())
        new_close_trade_event = handle_new_trade(trader_dict[key].close_trade_filter.get_new_entries())
        
        if new_sub_event:
            print("--------------------------")
            print(new_sub_event)
            print("--------------------------")
            sub_df = sub_df.append(new_sub_event[0]['args'], ignore_index=True)
        if new_trade_event:
            print("--------------------------")
            print((new_trade_event))
            print("--------------------------")
            all_trades_df = all_trades_df.append(new_trade_event[0]['args'], ignore_index=True)
        if new_close_trade_event:
            new_close_trade_event[0]["TraderId"] = key 
            print("--------------------------")
            print((new_close_trade_event))
            print("--------------------------")
            
    
    time.sleep(3)
    
atexit.register( exit_protocol, [(logger_df), (sub_df), (all_trades_df)] )

Starting while loop...


KeyboardInterrupt: 

In [136]:
all_trades_df

,TraderId,inputTraderAddress,tradeNum,inputOpen,inputSymbol,inputSize,inputEntryPriceEntryTime,inputExitPriceEntryTime,optionsData
0,2,0x902A693035257b8f1e999E4a2414c450F5cB9E03,9,True,AAPL,300,150 8/4/2022,NaN,none
1,2,0x902A693035257b8f1e999E4a2414c450F5cB9E03,10,True,AAPL,200,145.67 8/4/2022,NaN,none


In [134]:
new_close_trade_event

[]

In [128]:

trader_dict

{1: <__main__.Grader at 0x214018b8f88>, 2: <__main__.Grader at 0x214018b8908>}

In [120]:
test_df = pd.read_csv("Libraries/sub.csv")
test_df
test_df.drop(columns = [c for c in test_df.columns if "Unnamed" in c])

,TraderId,subNum,subAddr
0,8,0,0x902A693035257b8f1e999E4a2414c450F5cB9E03


In [ ]:
def load_trader_dict(df):
    df = drop_unnamed(df)
    try:
        df_as_dict = df.set_index('TraderId').to_dict()
        for key in df_as_dict['newLogAddress']:
            trader_dict[key] = Grader(key, df_as_dict['newLogAddress'][key])
    except KeyError:
        print("There was an issue with the logger.csv file. Please check the column names and make sure TraderId is there.")
    return trader_dict

In [133]:
test_df = pd.read_csv("Libraries/logger.csv")
test_df = test_df.drop(columns = [c for c in test_df.columns if "Unnamed" in c])
test_df = test_df.set_index('TraderId')
test_dict = test_df.to_dict()
other = {}
for key in test_dict['newLogAddress']:
    other[key] = Grader(key, test_dict['newLogAddress'][key])
int(test_df.value_counts())

1

In [97]:
all_trades_df

,TraderId,inputTraderAddress,tradeNum,inputOpen,inputSymbol,inputSize,inputEntryPriceEntryTime,inputExitPriceEntryTime,optionsData
0,2,0x902A693035257b8f1e999E4a2414c450F5cB9E03,7,True,MSFT,500,250 8/4/2022,NaN,none
1,2,0x902A693035257b8f1e999E4a2414c450F5cB9E03,8,True,AAPL,300,150 8/4/2022,NaN,none


In [67]:
dict = {'TraderId': 2, 'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03', 'tradeNum': 0, 'inputOpen': True, 'inputSymbol': 'MSFT', 'inputSize': '500', 'inputEntryPriceEntryTime': '250 8/4/2022', 'optionsData': 'none'}
list(dict.keys())

['TraderId',
 'inputTraderAddress',
 'tradeNum',
 'inputOpen',
 'inputSymbol',
 'inputSize',
 'inputEntryPriceEntryTime',
 'optionsData']

In [63]:
sub_df

,TraderId,subNum,subAddr
0,1,33,0x902A693035257b8f1e999E4a2414c450F5cB9E03
1,1,34,0x902A693035257b8f1e999E4a2414c450F5cB9E03
2,1,35,0x902A693035257b8f1e999E4a2414c450F5cB9E03
3,1,36,0x902A693035257b8f1e999E4a2414c450F5cB9E03


In [57]:
sub_df.columns = event[0]['args'].keys()
sub_df.append(event[0]['args'], ignore_index=True)
col_names = sub_df.columns.to_list()
col_names

['TraderId', 'subNum', 'subAddr']

In [52]:
event[0]['args'].keys()

dict_keys(['TraderId', 'subNum', 'subAddr'])

In [24]:
event = [{'args': {'TraderId': 1, 'subNum': 10, 'subAddr': '0x902A693035257b8f1e999E4a2414c450F5cB9E03'}, 'event': 'newSub', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0x4423e211e72cc7e2d12e79aece49ee76cd1a216cf98c4aa03b8457da92af9077', 'address': '0x808Ec94452e7B58ab2fbacd17772431bfae07fb8', 'blockHash': '0x752e7475c5cf17c5e955a34a952ae557b7e43462f0cd265815720715a6a89e15', 'blockNumber': 126}]

In [45]:
sub_df.columns=(["traderID","subNum","subAddress"])

""


In [40]:
list(event[0]['args'].values())

[1, 10, '0x902A693035257b8f1e999E4a2414c450F5cB9E03']

In [ ]:
trader_dict[13].close_trade_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x000001E7A0C081F8>,
 'eth_module': <web3.eth.Eth at 0x1e7a3a04d88>,
 'filter_id': '0x1d5',
 'callbacks': [],
 'filter_params': {'topics': ('0xa633b3cf9cf604b062687d6d28d6de58166d133b1709e63b10f3f893986f6292',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0xd032b99d5863e39F777Fc40ED0276f84888DCAbD'},
 'data_filter_set': (('uint256', None), ('uint256', None), ('string', None)),
 'data_filter_set_function': <function match_fn at 0x000001E7A0C0DCA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1e7a3ad9908>}

In [ ]:
new_log_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x00000162584D51F8>,
 'eth_module': <web3.eth.Eth at 0x1625b2d5d48>,
 'filter_id': '0x1b3',
 'callbacks': [],
 'filter_params': {'topics': ('0x3ccb4987690c8e37ec3a000310e45fbcef3730cf6068d27d5c688f294138287b',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0x4971CE19A9d9f57941C4C3A00ffE69d06746073b'},
 'data_filter_set': (('uint256', None), ('address', None)),
 'data_filter_set_function': <function match_fn at 0x00000162584DACA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1625b664a48>}

In [ ]:
trader_dict[key].new_trade_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x00000162584D51F8>,
 'eth_module': <web3.eth.Eth at 0x1625b2d5d48>,
 'filter_id': '0x1b5',
 'callbacks': [],
 'filter_params': {'topics': ('0xd31221654ebdc5f164b905fcf4b03959974549d0c7d04cf98383f5775b64b907',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0x86E6A9885fee00874AcCD6903eD307716c9181DD'},
 'data_filter_set': (('uint256', None),
  ('address', None),
  ('uint256', None),
  ('bool', None),
  ('string', None),
  ('string', None),
  ('string', None),
  ('string', None)),
 'data_filter_set_function': <function match_fn at 0x00000162584DACA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1625b6c5ac8>}

In [ ]:
trader_dict[2].__dict__


{'trader_id': 2,
 'log_address': '0xEe1Dcb4Ed69Ff562d0A546aa66F8d900A7961BC2',
 'contract': <web3._utils.datatypes.Contract at 0x1625b5e2948>,
 'new_trade_filter': <web3._utils.filters.LogFilter at 0x1625b57b248>}

In [ ]:
for key in trader_dict.keys():
    print(trader_dict[key].log_address)

In [ ]:
log_contract = load_contract_new(trader_dict[10].log_address)
event_filter = log_contract.events.newTrade.createFilter(fromBlock="latest")
event = handle_new_trade(event_filter.get_all_entries())
event

[{'args': {'TraderId': 10,
   'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03',
   'tradeNum': 0,
   'inputOpen': True,
   'inputSymbol': 'MSFT',
   'inputSize': 100,
   'inputFractional_shares': 0,
   'inputEntryPrice': 270,
   'inputExitPrice': 0,
   'inputExpirationTimeStamp': 0,
   'inputStrike': 0,
   'inputIsCall': True},
  'event': 'newTrade',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': '0x81c3a64878b1715ec220e47912b20a19bb9be569b1fd5e84afdd08c5d447abd3',
  'address': '0xa51183B1aeBf945b99B9A669fBbF0A0a617d6008',
  'blockHash': '0xf37c55755000731c3fedd4338bd5408660fb0299963d922fcf8028ba57e2eec4',
  'blockNumber': 49}]

In [ ]:

contract = load_contract() 

event_filter = contract.events.newTrade.createFilter(fromBlock="latest")

while True:
    event = handle_new_trade(event_filter.get_new_entries())

    print((event))
    print("--------------------------")
    print(type(event))
    time.sleep(10)

[]
--------------------------
<class 'list'>


In [ ]:
event

{'args': {'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03',
  'TradeId': 6,
  'inputOpen': True,
  'inputSymbol': 'MSFT',
  'inputSize': 100,
  'inputFractional_shares': 0,
  'inputEntryPrice': 260,
  'inputExitPrice': 0,
  'inputExpirationTimeStamp': 0,
  'inputStrike': 0,
  'inputIsCall': True},
 'event': 'newTrade',
 'logIndex': 0,
 'transactionIndex': 0,
 'transactionHash': '0x4495bd80668a1894602a25eb47f6552d466de87881e8cbf1524ed930bd307287',
 'address': '0xD6e7211B6199C64C7fd3578761dCC4b14B2Ec12d',
 'blockHash': '0xb21554d0825169cc5cb963ef3bc709313f80a55e89ca7db53383618a51a39e67',
 'blockNumber': 35}

In [ ]:
newLog = [{'args': {'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}, 'event': 'logCreated', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0xe79708588ce0cd9efea795fc942a5df83b00707fd8281b1b8d2f2cb22b8a5a32', 'address': '0x65570CFF945218356882cE36F3bEE72aca300206', 'blockHash': '0xf30c42450e1614851d85d99226ebdd4c91f565409ec0e78485f7f894a835b32e', 'blockNumber': 42}]

In [ ]:
new_log  = newLog[0]['args']
new_log

{'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}

In [ ]:
trader_dict = {}
trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'],new_log['newLogAddress'])

In [ ]:
trader_dict[8].log_address

'0x809Fb2c2E74f238dA88b58aBC598e8872195Ca58'

In [ ]:
def load_contract_new(log_addr):
    contract_address = log_addr
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
    address=contract_address,
    abi=logger_abi
        )
    return logger_contract

In [ ]:
ex_contract = load_contract_new(trader_dict[8].log_address)

In [ ]:
ex_contract